In [1]:
import pandas as pd
import numpy as np

In [40]:
orders = pd.read_csv('../../data/instacart_2017_05_01/orders.csv')
products = pd.read_csv('../../data/instacart_2017_05_01/products.csv')
order_products = pd.read_csv('../../data/instacart_2017_05_01/order_products__prior.csv')

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## Calculate average number of items per order, by day of week

In [5]:
item_counts = order_products.groupby('order_id').size()

In [6]:
orders_indexed = orders.set_index('order_id')

In [7]:
order_items = pd.concat([orders_indexed, item_counts], axis=1)

In [8]:
new_columns = order_items.columns.values
new_columns[6] = 'item_count'
order_items.columns = new_columns

In [9]:
order_items.head()

,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,item_count
order_id,,,,,,,
1,112108,train,4,4,10,9.0,NaN
2,202279,prior,3,5,9,8.0,9.0
3,205970,prior,16,5,17,12.0,8.0
4,178520,prior,36,1,9,7.0,13.0
5,156122,prior,42,6,16,9.0,26.0


In [39]:
order_items.groupby('order_dow').mean()['item_count']

order_dow
0    11.132983
1    10.177484
2     9.543501
3     9.321331
4     9.439436
5     9.881950
6    10.744480
Name: item_count, dtype: float64

# Most popular item by day/time of order

In [10]:
orders_indexed.order_dow.head()

order_id
2539329    2
2398795    3
473747     3
2254736    4
431534     4
Name: order_dow, dtype: int64

In [50]:
order_time = orders_indexed[['order_dow', 'order_hour_of_day']]

In [51]:
order_time.head()

,order_dow,order_hour_of_day
order_id,,
2539329,2,8
2398795,3,7
473747,3,12
2254736,4,7
431534,4,15


In [52]:
order_products_time = order_products.merge(order_time, left_on='order_id', right_index=True)

In [53]:
order_products_time.head()

,order_id,product_id,add_to_cart_order,reordered,order_dow,order_hour_of_day
0,2,33120,1,1,5,9
1,2,28985,2,1,5,9
2,2,9327,3,0,5,9
3,2,45918,4,1,5,9
4,2,30035,5,0,5,9


In [62]:
product_counts = order_products_time.groupby(['order_dow', 'order_hour_of_day', 'product_id']).count()['order_id']

In [63]:
product_counts.head(20)

order_dow  order_hour_of_day  product_id
0          0                  1              1
                              10             3
                              23             1
                              26             1
                              34             4
                              35             1
                              45            29
                              47             1
                              49             3
                              50             2
                              53             2
                              54             1
                              63             2
                              66             1
                              69             1
                              79             4
                              93             3
                              95             4
                              99             2
                              100            1
Name: order_id, dty

In [66]:
product_counts.groupby(level=[0,1]).nlargest(1)

order_dow  order_hour_of_day  order_dow  order_hour_of_day  product_id
0          0                  0          0                  24852          560
           1                  0          1                  24852          321
           2                  0          2                  24852          168
           3                  0          3                  24852          106
           4                  0          4                  24852           98
           5                  0          5                  24852          163
           6                  0          6                  24852          603
           7                  0          7                  24852         2214
           8                  0          8                  24852         4937
           9                  0          9                  24852         6916
           10                 0          10                 24852         8075
           11                 0          11                 

In [46]:
products[products.product_id == 21903]

,product_id,product_name,aisle_id,department_id
21902,21903,Organic Baby Spinach,123,4


## Most popular *first* item in an order

In [67]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [68]:
first_products = order_products[order_products.add_to_cart_order == 1]

In [69]:
first_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
9,3,33754,1,1
17,4,46842,1,0
30,5,13176,1,1
56,6,40462,1,0


In [70]:
first_products_time = first_products.merge(order_time, left_on='order_id', right_index=True)

In [71]:
first_products_time.head()

,order_id,product_id,add_to_cart_order,reordered,order_dow,order_hour_of_day
0,2,33120,1,1,5,9
9,3,33754,1,1,5,17
17,4,46842,1,0,1,9
30,5,13176,1,1,6,16
56,6,40462,1,0,1,12


In [83]:
product_counts = first_products_time.groupby(['order_dow', 'order_hour_of_day', 'product_id']).count()['order_id']

In [84]:
most_popular = product_counts.groupby(level=[0,1]).nlargest(3)

In [82]:
first_products_time = first_products_time.merge(products, left_on='product_id', right_on='product_id')

In [86]:
most_popular.index

MultiIndex(levels=[[0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [13176, 16797, 19660, 21137, 21903, 24852, 27845, 47209, 47766]],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3